<a href="https://colab.research.google.com/github/garinho/CAS_ML4/blob/main/sand_mine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install the libraries
!pip install earthengine-api geemap
import ee
import ee
import geemap


In [2]:
# Authenticate Google Earth Engine
ee.Authenticate()

In [3]:
# Initialize Earth Engine
ee.Initialize(project='ee-sandmining')

In [10]:
import ee
import geemap

# Initialize Earth Engine (only needed if not running in the GEE Code Editor)
ee.Initialize()

###############################################################################
# 1. Define overall bounding boxes for each region (focus on coastline)
###############################################################################
less_impact_areas = {
    'AlHoceima': ee.Geometry.Rectangle([-4.0, 35.0, -3.8, 35.2]),
    'CapSpartel': ee.Geometry.Rectangle([-5.95, 35.75, -5.85, 35.85]),
    'DakhlaPeninsula': ee.Geometry.Rectangle([-16.0, 23.6, -15.8, 23.8]),
    'OualidiaLagoon': ee.Geometry.Rectangle([-9.1, 32.65, -8.95, 32.75]),
    'Asilah': ee.Geometry.Rectangle([-6.10, 35.40, -5.95, 35.50]),
    'SidiKaouki': ee.Geometry.Rectangle([-9.85, 31.30, -9.70, 31.45]),
    'KhnifissLagoon': ee.Geometry.Rectangle([-12.4, 27.90, -12.2, 28.10])
}

###############################################################################
# 2. Cloud masking function (Sentinel-2 QA60 band)
###############################################################################
def maskS2clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10  # Clouds
    cirrus_bit_mask = 1 << 11 # Cirrus
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
        qa.bitwiseAnd(cirrus_bit_mask).eq(0)
    )
    return image.updateMask(mask).divide(10000)

###############################################################################
# 3. Helper to name exports based on region geometry
###############################################################################
def get_coordinates_str(bounds):
    coords = bounds.getInfo()['coordinates'][0]
    lon = round(coords[0][0], 3)
    lat = round(coords[0][1], 3)
    return f"{abs(lon):.3f}{'W' if lon < 0 else 'E'}_{abs(lat):.3f}{'N' if lat > 0 else 'S'}"

###############################################################################
# 4. Export function
###############################################################################
def export_images_to_gcp(region_name, bounds, start_date, end_date, description_suffix):
    dataset = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
               .filterBounds(bounds)
               .filterDate(start_date, end_date)
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
               .map(maskS2clouds)
               .median())

    coords_str = get_coordinates_str(bounds)
    export_desc = f'{region_name}_{coords_str}_{description_suffix}'

    task = ee.batch.Export.image.toCloudStorage(
        image=dataset,
        description=export_desc,
        bucket='ee-sandmining-images',  # Replace with your bucket name
        fileNamePrefix=export_desc,
        region=bounds,
        scale=10,  # 10m resolution
        maxPixels=1e13
    )
    task.start()
    print(f"Export started: {export_desc}")

###############################################################################
# 5. Main: For each region, split bounding box into 2 sub-boxes and export
###############################################################################
start_year = 2018
end_year = 2024  # Adjust if you only want up to 2023

# Each region's bounding box will be split into 2 sub-rectangles (2 rows × 1 col)
rows = 2
cols = 1

for region_name, total_bounds in less_impact_areas.items():
    # Retrieve bounding box coordinates
    coords = total_bounds.coordinates().getInfo()[0]
    west, south = coords[0]  # bottom-left corner
    east, north = coords[2]  # top-right corner

    lat_min, lat_max = south, north
    lon_min, lon_max = west, east

    lat_step = (lat_max - lat_min) / rows
    lon_step = (lon_max - lon_min) / cols

    sub_boxes = {}
    box_id = 1

    for r in range(rows):
        for c in range(cols):
            sub_west = lon_min + c * lon_step
            sub_east = lon_min + (c + 1) * lon_step
            sub_south = lat_min + r * lat_step
            sub_north = lat_min + (r + 1) * lat_step

            sub_geom = ee.Geometry.Rectangle([sub_west, sub_south, sub_east, sub_north])
            box_label = f'{region_name}_Box_{box_id}'
            sub_boxes[box_label] = sub_geom
            box_id += 1

    # Export each box in 6-month intervals
    for box_label, box_geom in sub_boxes.items():
        for year in range(start_year, end_year):
            # First half of the year
            export_images_to_gcp(
                region_name=box_label,
                bounds=box_geom,
                start_date=f'{year}-01-01',
                end_date=f'{year}-06-30',
                description_suffix=f'{year}_H1'
            )
            # Second half of the year
            export_images_to_gcp(
                region_name=box_label,
                bounds=box_geom,
                start_date=f'{year}-07-01',
                end_date=f'{year}-12-31',
                description_suffix=f'{year}_H2'
            )

###############################################################################
# 6. Optional: Visualize the split boxes for one region
###############################################################################
preview_region = 'AlHoceima'  # Choose any from the dictionary
preview_map = geemap.Map()
preview_map.centerObject(less_impact_areas[preview_region], 10)

# Rebuild sub-boxes for the preview
coords = less_impact_areas[preview_region].coordinates().getInfo()[0]
w, s = coords[0]
e, n = coords[2]
lat_min, lat_max = s, n
lon_min, lon_max = w, e

lat_step = (lat_max - lat_min) / rows
lon_step = (lon_max - lon_min) / cols
box_id = 1
for r in range(rows):
    for c in range(cols):
        sub_west = lon_min + c * lon_step
        sub_east = lon_min + (c + 1) * lon_step
        sub_south = lat_min + r * lat_step
        sub_north = lat_min + (r + 1) * lat_step
        geom = ee.Geometry.Rectangle([sub_west, sub_south, sub_east, sub_north])
        preview_map.addLayer(geom, {}, f'{preview_region}_Box_{box_id}')
        box_id += 1

preview_map


Export started: AlHoceima_Box_1_4.000W_35.000N_2018_H1
Export started: AlHoceima_Box_1_4.000W_35.000N_2018_H2
Export started: AlHoceima_Box_1_4.000W_35.000N_2019_H1
Export started: AlHoceima_Box_1_4.000W_35.000N_2019_H2
Export started: AlHoceima_Box_1_4.000W_35.000N_2020_H1
Export started: AlHoceima_Box_1_4.000W_35.000N_2020_H2
Export started: AlHoceima_Box_1_4.000W_35.000N_2021_H1
Export started: AlHoceima_Box_1_4.000W_35.000N_2021_H2
Export started: AlHoceima_Box_1_4.000W_35.000N_2022_H1
Export started: AlHoceima_Box_1_4.000W_35.000N_2022_H2
Export started: AlHoceima_Box_1_4.000W_35.000N_2023_H1
Export started: AlHoceima_Box_1_4.000W_35.000N_2023_H2
Export started: AlHoceima_Box_2_4.000W_35.100N_2018_H1
Export started: AlHoceima_Box_2_4.000W_35.100N_2018_H2
Export started: AlHoceima_Box_2_4.000W_35.100N_2019_H1
Export started: AlHoceima_Box_2_4.000W_35.100N_2019_H2
Export started: AlHoceima_Box_2_4.000W_35.100N_2020_H1
Export started: AlHoceima_Box_2_4.000W_35.100N_2020_H2
Export sta

Map(center=[35.10000016550748, -3.9000000000007202], controls=(WidgetControl(options=['position', 'transparent…